<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/master/notebooks/build_index_and_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id "10LV9QbZOkUyOzR4nh8hxesoKJhpmvpM9"   # citation vectors
!gdown --id "1-23aNm7j0bnycvyd_OaQfofVYPTewgOI"   # abstract vectors
!gdown --id "1-1HED-B-HtuZR9kNtybz9sypVpk2A_fX"   # TitlesAbstractsEmbedIds
!wget 'https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar'
!tar -xvf 'scibert_scivocab_uncased.tar'

Downloading...
From: https://drive.google.com/uc?id=10LV9QbZOkUyOzR4nh8hxesoKJhpmvpM9
To: /content/CitationSimilarityVectors106Epochs.npy
2.59GB [00:25, 99.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-23aNm7j0bnycvyd_OaQfofVYPTewgOI
To: /content/AbstractSimVectors.npy
2.59GB [00:39, 65.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-1HED-B-HtuZR9kNtybz9sypVpk2A_fX
To: /content/TitlesAbstractsEmbedIds.json.gzip
397MB [00:04, 96.0MB/s]
--2019-12-29 05:33:47--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.229.8
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.229.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 442460160 (422M) [application/x-tar]
Saving to: ‘scibert_scivocab_uncased.tar’

scibert_scivocab_un 100%[===================>] 421.96M  19.6MB/s    in 16s     



In [2]:
!pip install transformers --quiet

     |████████████████████████████████| 450kB 3.5MB/s 
     |████████████████████████████████| 1.0MB 52.5MB/s 
     |████████████████████████████████| 860kB 45.1MB/s 


In [3]:
%tensorflow_version 2.x
import numpy as np
import tensorflow as tf
from time import time
from tqdm import tqdm_notebook as tqdm
from transformers import BertTokenizer
import pandas as pd
from pprint import pprint

print('TensorFlow:', tf.__version__)

TensorFlow 2.x selected.
TensorFlow: 2.1.0-rc1


In [4]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  ['10.39.69.242:8470']
INFO:tensorflow:Initializing the TPU system: 10.39.69.242:8470


INFO:tensorflow:Initializing the TPU system: 10.39.69.242:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [5]:
workers = ['/job:worker/replica:0/task:0/device:TPU:'+str(i) for i in range(8)]
workers

['/job:worker/replica:0/task:0/device:TPU:0',
 '/job:worker/replica:0/task:0/device:TPU:1',
 '/job:worker/replica:0/task:0/device:TPU:2',
 '/job:worker/replica:0/task:0/device:TPU:3',
 '/job:worker/replica:0/task:0/device:TPU:4',
 '/job:worker/replica:0/task:0/device:TPU:5',
 '/job:worker/replica:0/task:0/device:TPU:6',
 '/job:worker/replica:0/task:0/device:TPU:7']

In [0]:
class Index:
    def __init__(self, embeddings, worker):
        self.embeddings = tf.math.l2_normalize(embeddings, axis=1)
        self.worker = worker

    @tf.function
    def search(self, query_vector):
      with tf.device(worker):
        dot_product = tf.reduce_sum(tf.multiply(self.embeddings, query_vector), axis=1)
        distances = 1 - dot_product
        sorted_indices =  tf.argsort(distances)
        nearest_distances = tf.gather(distances, sorted_indices)
        return nearest_distances[:20], sorted_indices[:20]

In [0]:
citations_embeddings = np.load('CitationSimilarityVectors106Epochs.npy')
abstract_embeddings = np.load('AbstractSimVectors.npy')
assert citations_embeddings.shape == abstract_embeddings.shape

In [9]:
# Discarding last 4 vectors to make number of vectors divisible by 8
citations_embeddings = np.split(citations_embeddings[:-4], 8, axis=0)
abstract_embeddings = np.split(abstract_embeddings[:-4], 8, axis=0)

vecs_per_index = citations_embeddings[0].shape[0]
print('Vectors per index :', vecs_per_index)

Vectors per index : 157874


In [10]:
## Place 1/8 of total embeddings on each TPU core
citation_indices = []
abstract_indices = []
for i, worker in enumerate(workers):
  with tf.device(worker):
    print('Building index with {} vectors on {}'.format(citations_embeddings[i].shape[0],worker))
    citation_indices.append(Index(citations_embeddings[i], worker))
    abstract_indices.append(Index(abstract_embeddings[i], worker))

Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:0
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:1
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:2
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:3
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:4
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:5
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:6
Building index with 157874 vectors on /job:worker/replica:0/task:0/device:TPU:7


In [0]:
def search(xq, top_k=10):
  cD, cI = [], []
  aD, aI = [], []
  for i in range(8):
    print('Search running on {}'.format(citation_indices[i].worker))
    cd, cidx = citation_indices[i].search(xq)
    ad, aidx = abstract_indices[i].search(xq)

    cD.extend(cd.numpy())
    aD.extend(ad.numpy())

    cI.extend(i*vecs_per_index + cidx.numpy())
    aI.extend(i*vecs_per_index + aidx.numpy())

  cid_sorted = np.argsort(cD)[:top_k]
  aid_sorted = np.argsort(aD)[:top_k]

  cD = np.array(cD)[cid_sorted]
  aD = np.array(aD)[aid_sorted]

  cI = np.array(cI)[cid_sorted]
  aI = np.array(aI)[aid_sorted]
  return cD, aD, cI, aI

In [0]:
model = tf.saved_model.load('gs://tfworld/saved_models')
tokenizer = BertTokenizer(vocab_file='scibert_scivocab_uncased/vocab.txt')

df = pd.read_json('TitlesAbstractsEmbedIds.json.gzip', compression = 'gzip')
embed2Title = pd.Series(df['title'].values,index=df['EmbeddingID']).to_dict()
embed2Abstract = pd.Series(df['paperAbstract'].values,index=df['EmbeddingID']).to_dict()

In [25]:
embed_id = 70
title = embed2Title[embed_id]
abstract = embed2Abstract[embed_id]

abstract_encoded = tokenizer.encode(abstract, max_length=512, pad_to_max_length=True)
abstract_encoded = tf.constant(abstract_encoded, dtype=tf.int32)[None, :]
print('Title : ')
pprint(title)
print('\nAbstract : ')
pprint(abstract)

Title : 
'Embedded operating systems energy overhead'

Abstract : 
("In this paper, a flow of characterization of embedded operating system's "
 'energy consumption is presented. The objective is to determine the energy '
 'overhead of the services of the embedded OS, we interest particularly on the '
 'context switch service. The modeling is based on measurements on the '
 'hardware platform OMAP35x EVM board, running Linux omap. Based on the '
 'analysis results, a relationship between energy overhead and a set of '
 'hardware and software parameters is established.')


In [32]:
s = time()
bert_output = model(abstract_encoded)
xq = tf.nn.l2_normalize(bert_output, axis=1)
e_p = time()

cD, aD, cI, aI = search(xq, top_k=5)
e_s = time()
print('\n'*2)
print('Prediction time  :', np.round(e_p-s, 3), 'secs')
print('Search time      :', np.round(e_s-e_p, 3), 'secs')
print('Total time       :', np.round(e_s - s, 3), 'secs')

print('\n'*2)
print('*'*80)
for i in range(len(cI)):
  print('Title : ')
  pprint(embed2Title[cI[i]])
  print('*'*80, )
print('\nNeighbours       :', cI )
print('Distances        :', np.round(cD, 4))

print('\n'*4)
print('*'*80)
for i in range(len(aI)):
  print('Abstract : ')
  pprint(embed2Abstract[aI[i]])
  print('*'*80)
print('\nNeighbours       :', aI )
print('Distances        :', np.round(aD, 4))

Search running on /job:worker/replica:0/task:0/device:TPU:0
Search running on /job:worker/replica:0/task:0/device:TPU:1
Search running on /job:worker/replica:0/task:0/device:TPU:2
Search running on /job:worker/replica:0/task:0/device:TPU:3
Search running on /job:worker/replica:0/task:0/device:TPU:4
Search running on /job:worker/replica:0/task:0/device:TPU:5
Search running on /job:worker/replica:0/task:0/device:TPU:6
Search running on /job:worker/replica:0/task:0/device:TPU:7



Prediction time  : 0.002 secs
Search time      : 0.964 secs
Total time       : 0.967 secs



********************************************************************************
Title : 
'Compiler-directed high-level energy estimation and optimization'
********************************************************************************
Title : 
('Power Measurement of Software Programs on Computers With Multiple I/O '
 'Components')
********************************************************************************
Title : 